In [ ]:
!pip3 install raster2xyz

In [2]:
import pandas as pd
import geopandas as gpd
import cdsapi
import xarray as xr
from urllib.request import urlopen
import datetime
import random
import cdsapi
import rasterio
from rasterio import features
from raster2xyz.raster2xyz import Raster2xyz
from shapely.geometry import Point
import pyproj
import requests
from bs4 import BeautifulSoup
import zipfile
import os
import glob
from rasterio.merge import merge

In [11]:
# URL of the page containing the zip file links
page_url = "https://descargas.icv.gva.es/CartografiaTridimensional2017_DATO_MUNICIPIO_ZIP"

# Send a GET request to the page
response = requests.get(page_url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Create a BeautifulSoup object to parse the HTML content
    soup = BeautifulSoup(response.content, "html.parser")

    # Find all the <a> tags that contain the zip file links
    link_tags = soup.find_all("a")

    # Extract the URLs from the <a> tags and filter them
    url_list = [link.get("href") for link in link_tags if link.get("href", "").startswith("/filebrowser/download/")]

    # Add the base URL to the filtered URLs
    base_url = "https://descargas.icv.gva.es"
    url_list = [base_url + url for url in url_list]

    # Print the list of URLs
    for url in url_list:
        print(url)
else:
    print("Failed to retrieve the page content.")


https://descargas.icv.gva.es/filebrowser/download/98192
https://descargas.icv.gva.es/filebrowser/download/98193
https://descargas.icv.gva.es/filebrowser/download/98194
https://descargas.icv.gva.es/filebrowser/download/98195
https://descargas.icv.gva.es/filebrowser/download/98196
https://descargas.icv.gva.es/filebrowser/download/98197
https://descargas.icv.gva.es/filebrowser/download/98198
https://descargas.icv.gva.es/filebrowser/download/98199
https://descargas.icv.gva.es/filebrowser/download/98200
https://descargas.icv.gva.es/filebrowser/download/98201
https://descargas.icv.gva.es/filebrowser/download/98202
https://descargas.icv.gva.es/filebrowser/download/98203
https://descargas.icv.gva.es/filebrowser/download/98204
https://descargas.icv.gva.es/filebrowser/download/98205
https://descargas.icv.gva.es/filebrowser/download/98206
https://descargas.icv.gva.es/filebrowser/download/98207
https://descargas.icv.gva.es/filebrowser/download/98208
https://descargas.icv.gva.es/filebrowser/downloa

In [12]:
len(url_list)

542

In [13]:
# Define the directory where you want to save the downloaded and extracted files
output_directory = "data/cartografia_tridimensional/municipios"

# Iterate through the URL list
for url in url_list:
    # Extract the filename from the URL
    filename = url.split("/")[-1]

    # Create the full path for the downloaded file
    file_path = os.path.join(output_directory, filename)

    # Send a GET request to download the file
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Save the file to the specified directory
        with open(file_path, "wb") as file:
            file.write(response.content)

        # Extract the contents of the downloaded zip file
        with zipfile.ZipFile(file_path, "r") as zip_ref:
            zip_ref.extractall(output_directory)

        # Remove the downloaded zip file
        os.remove(file_path)
    else:
        print(f"Failed to download file: {url}")


In [21]:
tif_files = glob.glob('data/cartografia_tridimensional/municipios/MDT_municipio/' + '*.tif')

In [22]:
tif_files

['data/cartografia_tridimensional/municipios/MDT_municipio/MDT_46066.tif',
 'data/cartografia_tridimensional/municipios/MDT_municipio/MDT_46072.tif',
 'data/cartografia_tridimensional/municipios/MDT_municipio/MDT_46099.tif',
 'data/cartografia_tridimensional/municipios/MDT_municipio/MDT_03062.tif',
 'data/cartografia_tridimensional/municipios/MDT_municipio/MDT_46258.tif',
 'data/cartografia_tridimensional/municipios/MDT_municipio/MDT_03076.tif',
 'data/cartografia_tridimensional/municipios/MDT_municipio/MDT_03089.tif',
 'data/cartografia_tridimensional/municipios/MDT_municipio/MDT_03116.tif',
 'data/cartografia_tridimensional/municipios/MDT_municipio/MDT_03102.tif',
 'data/cartografia_tridimensional/municipios/MDT_municipio/MDT_12027.tif',
 'data/cartografia_tridimensional/municipios/MDT_municipio/MDT_12033.tif',
 'data/cartografia_tridimensional/municipios/MDT_municipio/MDT_46112.tif',
 'data/cartografia_tridimensional/municipios/MDT_municipio/MDT_46106.tif',
 'data/cartografia_tridim

In [3]:
def merge_tif_files(directory, output_file):
    # Get a list of all .tif files in the directory
    tif_files = glob.glob(directory + '*.tif')

    if not tif_files:
        print("No .tif files found in the specified directory.")
        return

    # Read the .tif files
    datasets = []
    for file in tif_files:
        dataset = rasterio.open(file)
        datasets.append(dataset)

    # Merge the datasets
    try:
        merged_dataset, merged_transform = merge(datasets)
    except Exception as e:
        print("Error merging datasets:", str(e))
        return

    # Write the merged dataset to a new .tif file
    with rasterio.open(output_file, 'w', driver='GTiff', height=merged_dataset.shape[1], width=merged_dataset.shape[2], count=merged_dataset.shape[0], dtype=merged_dataset.dtype, crs='EPSG:3042', transform=merged_transform) as dst:
        dst.write(merged_dataset)

    # Close the dataset files
    for dataset in datasets:
        dataset.close()

    print("Merged file created: " + output_file)
    return merged_dataset

In [4]:
MDT_all = merge_tif_files('data/cartografia_tridimensional/municipios/MDT_municipio/', 'data/cartografia_tridimensional/MDT_all.tif')

In [15]:
MDT_all

array([[[-32767., -32767., -32767., ..., -32767., -32767., -32767.],
        [-32767., -32767., -32767., ..., -32767., -32767., -32767.],
        [-32767., -32767., -32767., ..., -32767., -32767., -32767.],
        ...,
        [-32767., -32767., -32767., ..., -32767., -32767., -32767.],
        [-32767., -32767., -32767., ..., -32767., -32767., -32767.],
        [-32767., -32767., -32767., ..., -32767., -32767., -32767.]]],
      dtype=float32)

In [5]:
type(MDT_all)

numpy.ndarray

In [6]:
MDT_all.shape

(1, 65669, 37807)

In [7]:
slope_all = merge_tif_files('data/cartografia_tridimensional/municipios/pendiente_municipio/', 'data/cartografia_tridimensional/pendiente_all.tif')

In [8]:
slope_all.shape

(1, 65669, 37807)

In [9]:
orientation_all = merge_tif_files('data/cartografia_tridimensional/municipios/orientacion_municipio/', 'data/cartografia_tridimensional/orientacion_all.tif')

In [10]:
orientation_all.shape

(1, 65669, 37807)

In [11]:
MDT_all_df = pd.DataFrame(MDT_all[0])
MDT_all_df

,0,1,2,3,4,5,6,7,8,9,...,37797,37798,37799,37800,37801,37802,37803,37804,37805,37806
0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,...,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0
1,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,...,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0
2,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,...,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0
3,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,...,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0
4,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,...,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65664,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,...,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0
65665,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,...,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0
65666,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,...,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0
65667,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,...,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0


In [12]:
slope_all_df = pd.DataFrame(slope_all[0])
slope_all_df

,0,1,2,3,4,5,6,7,8,9,...,37797,37798,37799,37800,37801,37802,37803,37804,37805,37806
0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,...,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0
1,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,...,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0
2,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,...,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0
3,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,...,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0
4,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,...,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65664,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,...,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0
65665,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,...,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0
65666,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,...,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0
65667,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,...,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0


In [13]:
orientation_all_df = pd.DataFrame(orientation_all[0])
orientation_all_df

,0,1,2,3,4,5,6,7,8,9,...,37797,37798,37799,37800,37801,37802,37803,37804,37805,37806
0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,...,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0
1,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,...,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0
2,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,...,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0
3,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,...,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0
4,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,...,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65664,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,...,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0
65665,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,...,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0
65666,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,...,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0
65667,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,...,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0,-32767.0


In [ ]:
# Get a list of all .tif files in the directory
tif_files = glob.glob('data/cartografia_tridimensional/municipios/MDT_municipio/' + '*.tif')

if not tif_files:
    print("No .tif files found in the specified directory.")
    raise

# Read the .tif files
rasters_df_list = []
for i, file in enumerate(tif_files):
    print(f'{i+1}/{len(tif_files)}')
    # Specify the paths to the GeoTIFF files (original coordinate system EPSG:3042)
    municipio = os.path.splitext(os.path.basename(file))[0].split("_")[-1]
    print('Municipio', municipio)
    elevation_file = f'data/cartografia_tridimensional/municipios/MDT_municipio/MDT_{municipio}.tif'
    slope_file = f'data/cartografia_tridimensional/municipios/pendiente_municipio/pendiente_{municipio}.tif'
    orientation_file = f'data/cartografia_tridimensional/municipios/orientacion_municipio/orientacion_{municipio}.tif'

    # Altitud
    elevation_csv = f'data/cartografia_tridimensional/datos_generados/altitud_{municipio}.csv'
    rtxyz = Raster2xyz()
    rtxyz.translate(elevation_file, elevation_csv)

    # Pendiente
    slope_csv = f'data/cartografia_tridimensional/datos_generados/pendiente_{municipio}.csv'
    rtxyz = Raster2xyz()
    rtxyz.translate(slope_file, slope_csv)

    # Orientacion
    orientation_csv = f'data/cartografia_tridimensional/datos_generados/orientacion_{municipio}.csv'
    rtxyz = Raster2xyz()
    rtxyz.translate(orientation_file, orientation_csv)

    elevation_df = pd.read_csv(elevation_csv)
    slope_df = pd.read_csv(slope_csv)
    orientation_df = pd.read_csv(orientation_csv)

    os.remove(elevation_csv)
    os.remove(slope_csv)
    os.remove(orientation_csv)

    elevation_df['altitud'] = elevation_df['z']
    slope_df['pendiente'] = slope_df['z']
    orientation_df['orientacion'] = orientation_df['z']

    elevation_df.drop('z', axis=1, inplace=True)
    slope_df.drop('z', axis=1, inplace=True)
    orientation_df.drop('z', axis=1, inplace=True)

    raster_df = elevation_df
    raster_df['pendiente'] = slope_df['pendiente']
    raster_df['orientacion'] = orientation_df['orientacion']
    raster_df.sample(100).to_csv(f'data/cartografia_tridimensional/datos_generados/raster_{municipio}.csv')
    # rasters_df_list.append(raster_df)

In [33]:
# Get a list of all .csv files in the directory
csv_files = glob.glob('data/cartografia_tridimensional/datos_generados/' + '*.csv')

if not csv_files:
    print("No .csv files found in the specified directory.")
    raise

# Create an empty list to store dataframes
dfs = []

# Iterate over the CSV files
for i, csv_file in enumerate(csv_files):
    print(f'{i+1}/{len(csv_file)}')
    # Read each CSV file into a dataframe
    df = pd.read_csv(csv_file)
    # Append the dataframe to the list
    dfs.append(df)

# Concatenate all dataframes into a single dataframe
rasters_df = pd.concat(dfs, ignore_index=True)

# Print the combined dataframe
print(rasters_df)
    

1/64
2/64
3/64
4/64
5/64
6/64
7/64
8/64
9/64
10/64
11/64
12/64
13/64
14/64
15/64
16/64
17/64
18/64
19/64
20/64
21/64
22/64
23/64
24/64
25/64
26/64
27/64
28/64
29/64
30/64
31/64
32/64
33/64
34/64
35/64
36/64
37/64
38/64
39/64
40/64
41/64
42/64
43/64
44/64
45/64
46/64
47/64
48/64
49/64
50/64
51/64
52/64
53/64
54/64
55/64
56/64
57/64
58/64
59/64
60/64
61/64
62/64
63/64
64/64
65/64
66/64
67/64
68/64
69/64
70/64
71/64
72/64
73/64
74/64
75/64
76/64
77/64
78/64
79/64
80/64
81/64
82/64
83/64
84/64
85/64
86/64
87/64
88/64
89/64
90/64
91/64
92/64
93/64
94/64
95/64
96/64
97/64
98/64
99/64
100/64
101/64
102/64
103/64
104/64
105/64
106/64
107/64
108/64
109/64
110/64
111/64
112/64
113/64
114/64
115/64
116/64
117/64
118/64
119/64
120/64
121/64
122/64
123/64
124/64
125/64
126/64
127/64
128/64
129/64
130/64
131/64
132/64
133/64
134/64
135/64
136/64
137/64
138/64
139/64
140/64
141/64
142/64
143/64
144/64
145/64
146/64
147/64
148/64
149/64
150/64
151/64
152/64
153/64
154/64
155/64
156/64
157/64
158/64
15

In [ ]:
# Append DataFrames
# rasters_df = pd.concat(rasters_df_list, ignore_index=True)

In [34]:
rasters_df.to_csv('data/cartografia_tridimensional/rasters_all.csv')

In [35]:
rasters_df

,Unnamed: 0,x,y,altitud,pendiente,orientacion
0,1853958,635421.882389,4.377432e+06,858.38650,3.307040,46.159070
1,3339844,630086.882389,4.374237e+06,824.54530,12.568801,238.782400
2,3372348,629716.882389,4.374167e+06,803.57750,56.137790,20.440338
3,1457899,633076.882389,4.378282e+06,761.31726,1.330915,213.325680
4,3443788,637866.882389,4.374017e+06,821.83875,13.170104,297.585140
...,...,...,...,...,...,...
54195,1023329,723439.305975,4.264740e+06,302.54474,4.820012,199.030330
54196,83393,725909.305975,4.268665e+06,603.67773,43.613365,314.474980
54197,1930334,724034.305975,4.260955e+06,214.98325,1.913605,259.227660
54198,2087976,727554.305975,4.260300e+06,171.72624,49.986637,170.423280


In [40]:
rasters_df.drop('Unnamed: 0', axis=1, inplace=True)

In [41]:
rasters_df

,x,y,altitud,pendiente,orientacion
0,635421.882389,4.377432e+06,858.38650,3.307040,46.159070
1,630086.882389,4.374237e+06,824.54530,12.568801,238.782400
2,629716.882389,4.374167e+06,803.57750,56.137790,20.440338
3,633076.882389,4.378282e+06,761.31726,1.330915,213.325680
4,637866.882389,4.374017e+06,821.83875,13.170104,297.585140
...,...,...,...,...,...
54195,723439.305975,4.264740e+06,302.54474,4.820012,199.030330
54196,725909.305975,4.268665e+06,603.67773,43.613365,314.474980
54197,724034.305975,4.260955e+06,214.98325,1.913605,259.227660
54198,727554.305975,4.260300e+06,171.72624,49.986637,170.423280


In [43]:
# Define the source and target coordinate systems
source_crs = pyproj.CRS('EPSG:3042')
target_crs = pyproj.CRS('EPSG:4326')

# Create a Proj transformer to convert coordinates
transformer = pyproj.Transformer.from_crs(source_crs, target_crs, always_xy=True)

# Reproject the coordinates to EPSG:4326
reprojected_geometry = []
for x, y in zip(rasters_df['x'], rasters_df['y']):
    lon, lat = transformer.transform(x, y)
    reprojected_geometry.append(Point(lon, lat))

# Create a GeoDataFrame with the reprojected geometry column
reprojected_gdf = gpd.GeoDataFrame(rasters_df, geometry=reprojected_geometry, crs=target_crs)

In [44]:
reprojected_gdf

,x,y,altitud,pendiente,orientacion,geometry
0,635421.882389,4.377432e+06,858.38650,3.307040,46.159070,POINT (-1.42417 39.53588)
1,630086.882389,4.374237e+06,824.54530,12.568801,238.782400,POINT (-1.48685 39.50793)
2,629716.882389,4.374167e+06,803.57750,56.137790,20.440338,POINT (-1.49117 39.50735)
3,633076.882389,4.378282e+06,761.31726,1.330915,213.325680,POINT (-1.45128 39.54390)
4,637866.882389,4.374017e+06,821.83875,13.170104,297.585140,POINT (-1.39644 39.50473)
...,...,...,...,...,...,...
54195,723439.305975,4.264740e+06,302.54474,4.820012,199.030330,POINT (-0.43765 38.50302)
54196,725909.305975,4.268665e+06,603.67773,43.613365,314.474980,POINT (-0.40808 38.53773)
54197,724034.305975,4.260955e+06,214.98325,1.913605,259.227660,POINT (-0.43204 38.46879)
54198,727554.305975,4.260300e+06,171.72624,49.986637,170.423280,POINT (-0.39194 38.46201)


In [46]:
reprojected_gdf_cleaned = reprojected_gdf.loc[reprojected_gdf['altitud'] > 0]

In [47]:
sample_df = reprojected_gdf_cleaned.sample(10000)

In [49]:
sample_df.to_file('data/cartografia_tridimensional/reprojected_municipios.geojson', driver='GeoJSON')